In [73]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

#Make the dictionary from sent_dic_ru file
with open('sent_dic_ru.txt') as f:
    sent_learn = f.readlines()
sent_learn = [x.strip('\n') for x in sent_learn]                
#print(sent_learn)

def norm(word):    
    p_obj = morph.parse(word)
    try:
        p = p_obj[0]
        return p.normal_form, p.tag.POS
    except IndexError:
        return None

In [74]:
def sent_learn_prep(sent):
    #Normalization and stop-words delete from sent_list
    sent_plus = []    
    label_plus = []
    sent_minus = []    
    label_minus = []

    sent_list = []
    for x in sent:      
        #Split into words
        str_list = x.split()

        #List from sentenses (words in list)
        sent_list.append(str_list)

    for sent in sent_list:
        #print(sent)
        one_sent = ''

        if abs(float(sent[-1])) > 0.3:
            for x in sent[:-1]:
                #print(x)
                x_norm = norm(x) 
                if x_norm is not None:
                    if x_norm[1] not in ['PREP', 'PRCL', 'CONJ', 'NPRO']:
                        one_sent = one_sent + ' ' + x_norm[0]                
            weight = float(sent[-1])
            if weight > 0:
                weight_norm = 1
            else:
                weight_norm = -1
            one_sent = one_sent.lstrip()
            #print(one_sent)
            if weight_norm > 0:                
                sent_plus.append(one_sent)
                label_plus.append(weight_norm)
            else:
                sent_minus.append(one_sent)
                label_minus.append(weight_norm)
    #print(sent_list_norm)
    #print(label_list)
    return (sent_plus, label_plus), (sent_minus, label_minus)

sent_p, sent_m = sent_learn_prep(sent_learn)
#print(sent_p, sent_m )    

In [75]:
from nltk.corpus import stopwords
stopWords = stopwords.words('russian')
#print(stopWords)

from sklearn.feature_extraction.text import CountVectorizer
vectoriz_p = CountVectorizer(stop_words = stopWords)
vectoriz_m = CountVectorizer(stop_words = stopWords)

#Bags of words making
sent_vect_p = vectoriz_p.fit_transform(sent_p[0])
sent_vect_m = vectoriz_m.fit_transform(sent_m[0])

#print(sent_vect_p)
#print(sent_vect_m)
#print(sent_vect_p.shape)
#print(sent_vect_m.shape)
#print(vectoriz_p.vocabulary_)
#print(vectoriz_m.vocabulary_)

import numpy as np

words_p = vectoriz_p.get_feature_names()
counts_p = np.asarray(sent_vect_p.sum(axis=0)).ravel()
#words_counts_p = map(lambda x,y: (x,y), words_p, counts_p)
#words_counts_p = zip(words_p, counts_p)
#print(*words_counts_p)
words_counts_p = dict(zip(words_p, counts_p))
#print(words_counts_p)

words_m = vectoriz_m.get_feature_names()
counts_m = np.asarray(sent_vect_m.sum(axis=0)).ravel()
#words_counts_p = map(lambda x,y: (x,y), words_p, counts_p)
#words_counts_m = zip(words_m, counts_m)
#print(*words_counts_m)
words_counts_m = dict(zip(words_m, counts_m))
#print(words_counts_m)

#Intersection between two sets (with +1 and -1 sentiment)
words_intersect = set(words_p).intersection(set(words_m))
#print(words_intersect)

In [76]:
#Expanding the list of stop words
for x in words_intersect:
    if abs(words_counts_p[x] - words_counts_m[x]) <= 1:
        stopWords.append(x)
#print(stopWords)
with open ('stopWords.txt', 'w') as fp:
    for x in stopWords:
        fp.write("%s\n" % (x))

In [77]:
#Making the joined bag of words
vectoriz = CountVectorizer(stop_words = stopWords, min_df = 0.001, max_df = 0.999)
sent_vect = vectoriz.fit_transform(sent_p[0] + sent_m[0])
#List of words - coordinates of the vector
words = vectoriz.get_feature_names()
counts = np.asarray(sent_vect.sum(axis=0)).ravel()
words_counts = dict(zip(words, counts))
#print(words_counts)
with open ('words_count.txt', 'w') as fp:
    for x in words_counts.keys():
        fp.write("%s %s\n" % (x, words_counts[x]))

In [78]:
#Making vectorizer for learning SVM (on the base of words list)
cv = CountVectorizer(vocabulary = words)
vect_sent = cv.fit_transform(sent_p[0] + sent_m[0])
vect_label = np.array(sent_p[1] + sent_m[1])

from sklearn.svm import SVC # "Support Vector Classifier"
clf = SVC(kernel='linear')
clf.fit(vect_sent, vect_label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [79]:
def sent_prep(sent):
    #Normalization and stop-words delete from sent_list
    sent_list_norm = []
    sent_list = []
    for x in sent:
        
        #Split into words
        str_list = x.split()

        #List from sentenses (words in list)
        sent_list.append(str_list)
    for sent in sent_list:
        #print(sent)
        one_sent = ''        
        for x in sent[:-1]:
                #print(x)
                x_norm = norm(x)
                if x_norm:                        
                    if x_norm[1] not in ['PREP', 'PRCL', 'CONJ', 'NPRO']:
                        one_sent = one_sent + ' ' + x_norm[0]
        one_sent = one_sent.lstrip()
        #print(one_sent)
        sent_list_norm.append(one_sent)                
    return sent_list_norm

In [80]:
import re

def raw_cut(text):
    #Delete digits    
    text = re.sub(r"\b\d+\b", "", text)
    #Split text into sentences and remove empty elements from list
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    sentences = list(filter(None, sentences))
    return sentences

In [81]:
text = input("Please enter some text: ")
sentences = raw_cut(text)
sent_list_norm = sent_prep(sentences)
#cv = CountVectorizer(vocabulary = dic)
vect_sent = cv.fit_transform(sent_list_norm)
#print(vect_sent)
vect_label_test = []

for x in vect_sent:
    vect_label_test.extend(clf.predict(x).tolist())

for i in range(len(sentences)):
    print(sentences[i], vect_label_test[i])   

Please enter some text: Сегодня была отличная погода. Город погрузился в мрак и уныние, наступил зомби-апокалипсис. Давайте отпразднуем наши достижения! Нет повода не выпить текилы. Переговоры сорвались и несогласие сторон привело к вооруженному конфликту.
Сегодня была отличная погода. 1
Город погрузился в мрак и уныние, наступил зомби-апокалипсис. 1
Давайте отпразднуем наши достижения! Нет повода не выпить текилы. 1
Переговоры сорвались и несогласие сторон привело к вооруженному конфликту. -1
